<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Задача" data-toc-modified-id="Задача-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Задача</a></span><ul class="toc-item"><li><span><a href="#Общий-план-работы." data-toc-modified-id="Общий-план-работы.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Общий план работы.</a></span></li></ul></li><li><span><a href="#Загрузка-библиотек,-датасета,-знакомство-с-датасетом,-первичная-обработка-и-проверки." data-toc-modified-id="Загрузка-библиотек,-датасета,-знакомство-с-датасетом,-первичная-обработка-и-проверки.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка библиотек, датасета, знакомство с датасетом, первичная обработка и проверки.</a></span></li><li><span><a href="#Выгрузка-в-CSV" data-toc-modified-id="Выгрузка-в-CSV-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выгрузка в CSV</a></span></li></ul></div>

# Задача

Мы работаем аналитиком в Яндекс.Дзене. Почти всё время занимает анализ пользовательского взаимодействия с карточками статей.

Каждую карточку определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия».

Пользователей системы характеризует возрастная категория. Скажем, «26-30» или «45+».

Есть три способа взаимодействия пользователей с системой:
    
    - Карточка отображена для пользователя (show);
    - Пользователь кликнул на карточку (click);
    - Пользователь просмотрел статью карточки (view).

Каждую неделю начинающие менеджеры Денис и Валерия задают вам одни и те же вопросы:
    
    - Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
    - Как много карточек генерируют источники с разными темами?
    - Как соотносятся темы карточек и темы источников?

    На шестую неделю работы вы решаете, что процесс пора автоматизировать. Для Дениса и Валерии нужно сделать дашборд.
Дашборд будет основываться на пайплайне, который будет брать данные из таблицы, в которых хранятся сырые данные, трансформировать данные и укладывать их в агрегирующую таблицу. Пайплайн будет разработан для вас дата-инженерами.

    Пообщавшись с менеджерами и администраторами баз данных, вы написали краткое ТЗ:

    - Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;
    - Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
    - Кто будет основным пользователем дашборда: менеджеры по анализу контента;

    Состав данных для дашборда:
    - История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
    - Разбивка событий по темам источников;
    - Таблица соответствия тем источников темам карточек;

    По каким параметрам данные должны группироваться:
    - Дата и время;
    - Тема карточки;
    - Тема источника;
    - Возрастная группа;

    Характер данных:
    - История событий по темам карточек — абсолютные величины с разбивкой по минутам;
    - Разбивка событий по темам источников — относительные величины (% событий);
    - Соответствия тем источников темам карточек - абсолютные величины;
    - Важность: все графики имеют равную важность;

    Источники данных для дашборда: дата-инженеры обещали подготовить для вас агрегирующую таблицу dash_visits. Вот её структура:
    - record_id — первичный ключ,
    - item_topic — тема карточки,
    - source_topic — тема источника,
    - age_segment — возрастной сегмент,
    - dt — дата и время,
    - visits — количество событий.
    - Таблица хранится в специально подготовленной для вас базе данных zen;
    - Частота обновления данных: один раз в сутки, в полночь по UTC;

Какие графики должны отображаться и в каком порядке, какие элементы управления должны быть на дашборде отраженно в согласованном проекте дашборда.

## Общий план работы.
    
    - Мы конечно доверяем коллегам, но базовые проверки ДС никто не отменял.
    - загрузим ДС, изучим его, посмотрим на данные, добавим столбцы (которых нам будет нехватать) и/или скорректируем типы данным
    - сохраним датасет в формате csv и загрузим в Tableau
    - построим графики (в Tableau), которые будут отвечать на вопросы коллег, которые потом используем в презентации.
        - Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
        - Как много карточек генерируют источники с разными темами?
        - Как соотносятся темы карточек и темы источников?
    - после подготовки и перепроверки ДС, приступим к отрисовке дашборда для колллег, с учетом его согласованных параметров 

# Загрузка библиотек, датасета, знакомство с датасетом, первичная обработка и проверки.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

In [3]:
query = '''
            SELECT * FROM dash_visits
        '''

In [4]:
dash_visits = pd.io.sql.read_sql(query, con = engine)

In [5]:
dash_visits

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
...,...,...,...,...,...,...
30740,1071337,Юмор,Финансы,36-40,2019-09-24 18:57:00,2
30741,1071338,Юмор,Финансы,36-40,2019-09-24 19:00:00,1
30742,1071339,Юмор,Финансы,41-45,2019-09-24 18:54:00,1
30743,1071340,Юмор,Финансы,41-45,2019-09-24 18:56:00,1


In [6]:
dash_visits.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB


In [7]:
def check(data):  # функция поиска дубликатов
    try:
        
        display('Проверка на дубликаты:')
        duplicates = data.duplicated()
        duplicate_rows = data.loc[duplicates]
        display(duplicate_rows.info())
        display(duplicate_rows)
        display('----------------------')
        display('Пропуски:')
        display(data.isna().sum())
        display('Пропуски в процентном отношении к всему датасету:')
        display(data.isna().sum() / len(data) * 100)

        num_rows_before = len(data)
        data.drop_duplicates(inplace=True)
        num_rows_after = len(data)
        num_rows_deleted = num_rows_before - num_rows_after
        percent_deleted = round(num_rows_deleted / num_rows_before * 100, 2)
        display(f'Удалено дубликатов: {num_rows_deleted} строк ({percent_deleted}% от всего датасета)')
        
    except Exception as e:
        print(f'ERROR: {e}')

In [8]:
check (dash_visits)
# очевидных дубликатов нет 

'Проверка на дубликаты:'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     0 non-null      int64         
 1   item_topic    0 non-null      object        
 2   source_topic  0 non-null      object        
 3   age_segment   0 non-null      object        
 4   dt            0 non-null      datetime64[ns]
 5   visits        0 non-null      int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 0.0+ bytes


None

,record_id,item_topic,source_topic,age_segment,dt,visits


'----------------------'

'Пропуски:'

record_id       0
item_topic      0
source_topic    0
age_segment     0
dt              0
visits          0
dtype: int64

'Пропуски в процентном отношении к всему датасету:'

record_id       0.0
item_topic      0.0
source_topic    0.0
age_segment     0.0
dt              0.0
visits          0.0
dtype: float64

'Удалено дубликатов: 0 строк (0.0% от всего датасета)'

In [9]:
def check_unique(data):  # проверка на уникальность и оценка данных в датасете
    for col in data.select_dtypes(include=['object']):
        print(f"Уникальные значения в столбце {col}:")
        print(data[col].unique())
        print('---------------------')

    for col in data.select_dtypes(include=['datetime64']):
        print(f"Диапазон значений в столбце {col}:")
        print(f"Минимальное значение: {data[col].min()}")
        print(f"Максимальное значение: {data[col].max()}")
        print('---------------------')

    for col in data.select_dtypes(include=['int64', 'float64']):
        if len(data[col].unique()) > 10:
            print(f"В столбце {col} более 10 уникальных значений")
        else:
            print(f"Уникальные значения в столбце {col}:")
            print(data[col].unique())
        print('---------------------')

In [10]:
check_unique (dash_visits)

Уникальные значения в столбце item_topic:
['Деньги' 'Дети' 'Женская психология' 'Женщины' 'Здоровье' 'Знаменитости'
 'Интересные факты' 'Искусство' 'История' 'Красота' 'Культура' 'Наука'
 'Общество' 'Отношения' 'Подборки' 'Полезные советы' 'Психология'
 'Путешествия' 'Рассказы' 'Россия' 'Семья' 'Скандалы' 'Туризм' 'Шоу'
 'Юмор']
---------------------
Уникальные значения в столбце source_topic:
['Авто' 'Деньги' 'Дети' 'Еда' 'Здоровье' 'Знаменитости' 'Интерьеры'
 'Искусство' 'История' 'Кино' 'Музыка' 'Одежда' 'Полезные советы'
 'Политика' 'Психология' 'Путешествия' 'Ремонт' 'Россия' 'Сад и дача'
 'Сделай сам' 'Семейные отношения' 'Семья' 'Спорт' 'Строительство'
 'Технологии' 'Финансы']
---------------------
Уникальные значения в столбце age_segment:
['18-25' '26-30' '31-35' '36-40' '41-45' '45+']
---------------------
Диапазон значений в столбце dt:
Минимальное значение: 2019-09-24 18:28:00
Максимальное значение: 2019-09-24 19:00:00
---------------------
В столбце record_id более 10 уник

In [11]:
dash_visits ['item_topic'].nunique()

25

In [13]:
dash_visits ['source_topic'].nunique()

26

# Выгрузка в CSV
Выгрузим файл в csv для использования в Tableau

In [12]:
dash_visits.to_csv('visits_data.csv', index=False)